# Load libraries. Set precision and fonts

In [13]:
# reload
#%reset
%load_ext autoreload
%autoreload 2    
    
from helper_functions import *
from tabular_dae.model import DAE
from tabular_dae.model import load as dae_load
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer
from torch.optim import Adam

get_ipython().run_line_magic('precision', '3')
plt.rcParams.update({'font.size': 22})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("No GPU found.")
print(torch.cuda.device_count())
#print(torch.cuda.get_device_name(0))
#print(torch.cuda.current_device())

GPU is available.
1


# Set Experiment Parameters

In [18]:
#========================================
# Experiment Parameters: Check carefully!
#========================================

model_save_path = 'models/without_pretrain/'
train_slice = 'all' #['macro', 'micro', 'slow', 'fast', 'all', 'only_delay']
test_slice = 'all' #['macro', 'micro', 'slow', 'fast', 'all']
EXP_PARAM = {
    'scaler': 'standard' #'minmax', 'standard', 'robust', 'maxabs', 'l2norm'
} 
scaler_to_save_name = EXP_PARAM['scaler']+'_scaler'

#==================================
# Pretraining Experiment Parameters
#==================================

pretrain = False
pretrain_type = 'tabnet' #['dae', 'tabnet']
pretrain_model_to_save_name = train_slice+'_tabnet_default_hyperp'

#==========================================
# Supervised Training Experiment Parameters
#==========================================
# Load an existing sup model to evaluate 
load_sup_model = False
sup_model_to_load_type = 'tabnet'
sup_model_to_load_name = train_slice+'_tabnet_suptrain_no_pretrain'

# Load an existing pretrained model to use as encoding for sup model 
use_pretrained_model = False
pretrain_model_to_load_type = 'tabnet' #['dae', 'tabnet']
pretrain_model_to_load_name = 'all_pretrain_tabnet_default_hyperp.zip'
scaler_to_load_name = 'minMaxScaler_pretrain_tabnet'

# Train a sup model with or without using a pretrained model 
sup_model_type = 'xgb' #['mlp', 'xgb', 'tabnet']
suptrain_model_to_save_name = train_slice+'_xgb_suptrain_no_pretrain_standard_scaler' # could also be sup_model_with_pretrain 


#==================================================
# Experiment Parameters: Not often changed
#==================================================

pretrain_runs = range(1, 12 + 1) # 1-12
train_runs = range(1, 12 + 1)
test_runs = range(13, 15 + 1)

time_step_size = '500ms'
#past_sample_step_size = '10ms'
past_samples = 1

fill_na_val = 0

# Column names in the dataset that contain the ground truth labels for our prediction tasks of interest
all_learning_tasks_in_data = ['dashClient_trace.txt_newBitRate_bps', # predictable with 0.3516 mape # most imp feature DlPdcpStats.txt_PduSize
                              'dashClient_trace.txt_oldBitRate_bps', 
                              'delay_trace.txt_ul_delay', # predictable with 0.06 mape # most imp feature UlPdcpStats.txt_delay
                              'delay_trace.txt_dl_delay', # predictable with 0.07 mape # most imp feature DlPdcpStats.txt_delay
                              'vrFragment_trace.txt_vr_frag_time', # predictable with 0.0394 mape # most imp feature DlPdcpStats.txt_delay
                              'vrFragment_trace.txt_vr_frag_thput_mbps', # predictable with 0.1150 mape # most imp feature DlPdcpStats.txt_delay
                              'vrFragment_trace.txt_vr_burst_time', # predictable with 0.0869 mape # most imp feature DlPdcpStats.txt_max
                              'vrFragment_trace.txt_vr_burst_thput_mbps']# predictable with 0.2122 mape # most imp feature DlPdcpStats.txt_delay 
learning_tasks = ['vrFragment_trace.txt_vr_frag_thput_mbps', 'vrFragment_trace.txt_vr_burst_thput_mbps',
                  'vrFragment_trace.txt_vr_frag_time', 'vrFragment_trace.txt_vr_burst_time', 
                  'dashClient_trace.txt_newBitRate_bps']
                  #'delay_trace.txt_ul_delay',
                  #'delay_trace.txt_dl_delay']

classification = False 

# If True then we are predicting one window ahead if False then we are predicting on the same window 
shift_samp_for_predict = False

# If you want the test samples to be sorted by delay value to see the error differences for the low delay and high delay cases 
sort_test_samples = False

# CHECK
# Drop the other Y columns for other learning tasks
drop_cols = ['timestamp']

use_all_feats = True
# take the top n features of each run and add it to the top_n_features list  
# If use_all_feats = True then thes will not be used 
feat_filter = 10 
top_n_features = []
# Only valid when use_all_feats = False 
selected_features = []

if classification:
    IN_PARAM = IN_PARAM_C
    OUT_PARAM = OUT_PARAM_C
else:
    IN_PARAM = IN_PARAM_R
    OUT_PARAM = OUT_PARAM_R
    IN_PARAM['loss'] = 'mse' # options are mse, mae and mape 
    IN_PARAM['eval_metric'] = 'mape' # options are mae, and mape
    
IN_PARAM['model_type'] = sup_model_type
IN_PARAM['time_wind_size'] = time_step_size

# All delay values above this will be removed from the train and test set
clip_outliers = True
delay_clip_th = 5000

# Read these from the ue_groups file later and also rename ue_groups to something else
network_info={}
network_info['total_num_ues'] = 90
network_info['macro_cells'] = [1,2,3]
network_info['micro_cells'] = [4,5,6]
network_info['macro_imsis'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
network_info['fast_imsis'] = [1,2,3,4,5,6]
network_info['only_delay_imsis'] = [3,5,7,9,13,15,17,19,23,25,27,29,33,35,37,39,43,45,47,49,53,55,57,59,63,65,67,69,73,75,77,79,83,85,87]
network_info['micro_imsis'] = list(set(range(1, network_info['total_num_ues']+1)) - set(network_info['macro_imsis']))

# Set Model Hyperparameters

In [19]:
# Hyperparameters for supervised TabNet training
hypp_sup_tabnet={
    #'lambda_sparse': , # default = 1e-3
    # This is the extra sparsity loss coefficient as proposed in the original paper. The bigger this coefficient is, the sparser your model will be in terms of feature selection. Depending on the difficulty of your problem, reducing this value could help.
    'mask_type': 'sparsemax', # 'entmax' # default='sparsemax'
    'n_da': 8, # between 8-64 # default=8
    'n_steps': 3, # between 3-10 # default=3
    'n_independent': 2, # between 1-5 # default=2
    'n_shared': 2, # between 1-5 # default=2 
    'batch_size': 1024, #default=1024
    'max_epochs': 200, # default=200
    'patience': 10 # default=10
} 

# Hyperparameters for SSL using TabNet
hypp_ssl_tabnet={
    #'lambda_sparse': , # default = 1e-3
    # This is the extra sparsity loss coefficient as proposed in the original paper. The bigger this coefficient is, the sparser your model will be in terms of feature selection. Depending on the difficulty of your problem, reducing this value could help.
    'mask_type': 'entmax', # 'entmax' # default='sparsemax'
    'n_da': 8, # between 8-64 # default=8
    'n_steps': 3, # between 3-10 # default=3
    'n_independent': 2, # between 1-5 # default=2
    'n_shared': 2, # between 1-5 # default=2
    'n_shared_decoder': 1, # default=1
    'n_indep_decoder': 1, # default=1
    'noise_ratio': 0.30,
    'batch_size': 1024, #default=1024
    'max_epochs': 200, # default=200
    'patience': 10 # default=10
} 

# Hyperparameters for SSL using DAE 
hypp_ssl_dae={
    'swap_noise_prob': 0.50,
    'batch_size': 128,   

    # used if 'deepbottleneck'
    #'arch': 'deepbottleneck',
    #'hid_size': 150,
    #'bottle_neck_size': 40,
    #'num_layers': 5
    
    # used if 'deepstack'
    'arch': 'deepstack',
    'hid_size': 414,
    'num_layers': 3
} 

# Hyperparameters for supervised MLP training
hypp_sup_mlp={
    'fc_layers': [500, 100], # the hidden layers
    'batch_size': 128,  
    'max_epochs': 200,
    'patience': 10
} 
# Hyperparameters for supervised XGB training 
hypp_sup_xgb={
    
} 

if sup_model_type == 'tabnet':
    sup_hyper_params=hypp_sup_tabnet
elif sup_model_type == 'mlp':
    sup_hyper_params=hypp_sup_mlp
elif sup_model_type == 'xgb':
    sup_hyper_params=hypp_sup_xgb

# Process data frame before separating out learning task

In [20]:
# Set random seed
r = np.random.randint(0, 1000)
print('Run with random seed: ', r, '------------------------------------------------------------------------')    
IN_PARAM['rand_seed'] = r
np.random.seed(r)

# Read the dataset

if pretrain:
    pretrain_data = pd.DataFrame()
    print('Pretrain runs: ', pretrain_runs)
    for run in pretrain_runs:
        this_run_data = pd.read_csv('../../../dataset_ver1/parsed_data/run'+str(run)+'_dataslice_all_video_delay_vr_'+time_step_size+'.csv', delimiter=",")
        # separate the rows that belong to the desired data slice  
        this_run_data = data_slice_filter_samples(train_slice, this_run_data, network_info)
        pretrain_data = pd.concat([pretrain_data, this_run_data], axis=0)
        print('Loaded run', str(run))
        
    # Name the first column as timestamp
    pretrain_data.rename(columns={pretrain_data.columns[0]: 'timestamp'}, inplace=True)
    # Drop columns that we do not want to include in the dataset
    print('Drop unwanted columns')
    if use_all_feats:
        pretrain_data = pretrain_data.drop(expand_cols_to_step_size(drop_cols, past_samples), axis=1, errors='ignore')
    else:
        pretrain_data = pretrain_data[top_n_agg]
    print(pretrain_data.shape)


print('Train runs: ', train_runs)
train_data = pd.DataFrame()
for run in train_runs:
    this_run_data = pd.read_csv('../../../dataset_ver1/parsed_data/run'+str(run)+'_dataslice_all_video_delay_vr_'+time_step_size+'.csv', delimiter=",")
    # separate the rows that belong to the desired data slice  
    this_run_data = data_slice_filter_samples(train_slice, this_run_data, network_info)
    train_data = pd.concat([train_data, this_run_data], axis=0)
    print('Loaded run', str(run))
# Name the first column as timestamp
train_data.rename(columns={train_data.columns[0]: 'timestamp'}, inplace=True)
# Drop columns that we do not want to include in the dataset
print('Drop unwanted columns')
if use_all_feats:
    train_data = train_data.drop(expand_cols_to_step_size(drop_cols, past_samples), axis=1, errors='ignore')
else:
    train_data = train_data[top_n_agg]
print(train_data.shape)

print('Test runs: ', test_runs)
test_data = pd.DataFrame()
for run in test_runs:
    this_run_data = pd.read_csv('../../../dataset_ver1/parsed_data/run'+str(run)+'_dataslice_all_video_delay_vr_'+time_step_size+'.csv', delimiter=",")
    # separate the rows that belong to the desired data slice  
    this_run_data = data_slice_filter_samples(test_slice, this_run_data, network_info)
    test_data = pd.concat([test_data, this_run_data], axis=0)
    print('Loaded run', str(run))
# Name the first column as timestamp
test_data.rename(columns={test_data.columns[0]: 'timestamp'}, inplace=True)
# Drop columns that we do not want to include in the dataset
print('Drop unwanted columns')
if use_all_feats:
    test_data = test_data.drop(expand_cols_to_step_size(drop_cols, past_samples), axis=1, errors='ignore')
else:
    test_data = test_data[top_n_agg]
print(test_data.shape)


# Save the column names
#np.savetxt('all_columns_list.csv', np.array(dae_pretrain_data.columns), delimiter=',', fmt="%s")

Run with random seed:  865 ------------------------------------------------------------------------
Train runs:  range(1, 13)
Loaded run 1
Loaded run 2
Loaded run 3
Loaded run 4
Loaded run 5
Loaded run 6
Loaded run 7
Loaded run 8
Loaded run 9
Loaded run 10
Loaded run 11
Loaded run 12
Drop unwanted columns
(2158920, 215)
Test runs:  range(13, 16)
Loaded run 13
Loaded run 14
Loaded run 15
Drop unwanted columns
(539730, 215)


# Pretrain a new model

In [18]:
if pretrain:
    # Separate the X and the ys from the data
    # remove the labels of all prediction tasks which are also in the datset 
    X_pretrain = pretrain_data.drop(all_learning_tasks_in_data, axis=1, errors='ignore')
    print(X_pretrain.shape)
    
    # Fill with 0 the values that are missing in the input features so that the sample can still be used
    print('Filling NA in samples with ', fill_na_val)
    print('Number of rows with NA values in the input features: ', len(X_pretrain[X_pretrain.isna().any(axis=1)]))
    print('Fraction of rows with NA values in the input features: ', len(X_pretrain[X_pretrain.isna().any(axis=1)])/X_pretrain.shape[0])
    X_pretrain = X_pretrain.fillna(fill_na_val)
    print('NOT imputing NA values and just keeing them as NAs instead')
    
    val_scaler = create_scaler(X_pretrain, EXP_PARAM['scaler'])
    # Save the scaler for later use when doing supervised training 
    with open(model_save_path + scaler_to_save_name +'.pkl', 'wb') as f:
        pickle.dump(val_scaler, f)
    
    if pretrain_type == 'dae':
        pretrain_model = pretrain_with_dae(pd.DataFrame(val_scaler.transform(X_pretrain).copy()))
        # save this pretrained model for later use 
        pretrain_model.save(model_save_path+pretrain_model_to_save_name)
    elif pretrain_type == 'tabnet':
        X_pretrain, X_pretrain_val = train_test_split(X_pretrain, test_size=0.2, random_state=IN_PARAM['rand_seed'], shuffle=True) 
        pretrain_model = pretrain_with_tabnet(pd.DataFrame(val_scaler.transform(X_pretrain).copy()),  pd.DataFrame(val_scaler.transform(X_pretrain_val).copy()))
        pretrain_model.save_model(model_save_path+pretrain_model_to_save_name)
        # show the results of feature importance
        tabnet_explain(pretrain_model, X_pretrain_val)

(2158920, 207)
Filling NA in samples with  0
Number of rows with NA values in the input features:  109754
Fraction of rows with NA values in the input features:  0.0508374557649195
NOT imputing NA values and just keeing them as NAs instead


/home/ubuntu/.local/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 6465.10605| val_0_unsup_loss_numpy: 0.610480010509491|  0:02:10s
epoch 1  | loss: 0.82321 | val_0_unsup_loss_numpy: 0.4271799921989441|  0:04:19s
epoch 2  | loss: 0.58634 | val_0_unsup_loss_numpy: 0.42021000385284424|  0:06:27s
epoch 3  | loss: 0.50954 | val_0_unsup_loss_numpy: 0.363180011510849|  0:08:37s
epoch 4  | loss: 0.48055 | val_0_unsup_loss_numpy: 0.3509899973869324|  0:10:46s
epoch 5  | loss: 0.42669 | val_0_unsup_loss_numpy: 0.35231998562812805|  0:12:56s
epoch 6  | loss: 0.42188 | val_0_unsup_loss_numpy: 0.5714300274848938|  0:15:06s
epoch 7  | loss: 0.40899 | val_0_unsup_loss_numpy: 0.34619998931884766|  0:17:15s
epoch 8  | loss: 0.40128 | val_0_unsup_loss_numpy: 0.3446600139141083|  0:19:24s
epoch 9  | loss: 0.3885  | val_0_unsup_loss_numpy: 0.3690600097179413|  0:21:34s
epoch 10 | loss: 0.37973 | val_0_unsup_loss_numpy: 0.37501001358032227|  0:23:44s
epoch 11 | loss: 0.37538 | val_0_unsup_loss_numpy: 0.3400599956512451|  0:25:54s
epoch 12 | loss: 0.36892

/home/ubuntu/.local/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


# Load Tabnet pretrained model for explainability results

In [ ]:
# Load a pretrained model for getting explainability results
# Load the pretrained model and scaler 
#with open(model_save_path + scaler_to_load_name + '.pkl', 'rb') as f:
#    val_scaler = pickle.load(f)
#
#
## remove the labels of all prediction tasks which are also in the datset 
#X_pretrain_val = test_data.drop(all_learning_tasks_in_data, axis=1, errors='ignore')
#print(X_pretrain_val.shape)
#
## Fill with 0 the values that are missing in the input features so that the sample can still be used
#X_pretrain_val = X_pretrain_val.fillna(fill_na_val)
#
#X_pretrain_val = val_scaler.transform(X_pretrain_val).copy()
# 
#if pretrain_model_to_load_type == 'tabnet':
#    pretrain_model = TabNetPretrainer()
#    pretrain_model.load_model(model_save_path+pretrain_model_to_load_name)
#    X_pretrain_val = pretrain_model.predict(X_pretrain_val)[0]
#    tabnet_explain(pretrain_model, X_pretrain_val)
#else:
#    print('Do not have explainability functionality for this model type')

# Train and test model 

In [21]:
for learning_task in learning_tasks:
    
    print('Learning task: ', learning_task)

    # Prepare the train and test sets
    
    # Drop rows when the label or ground truth is NA 
    print('Dropping rows for which the label is NA, since there is no ground truth')
    train_data_na_dropped = train_data.dropna(subset=[learning_task])
    test_data_na_dropped = test_data.dropna(subset=[learning_task])
    print(train_data_na_dropped.shape)
    print(test_data_na_dropped.shape)
    
    # Separate the X and the y from the data
    # separate out the prediction task label column 
    y_train = train_data_na_dropped[learning_task]
    y_test = test_data_na_dropped[learning_task]
    # remove the labels of this and other prediction tasks which are also in the datset 
    X_train = train_data_na_dropped.drop(all_learning_tasks_in_data, axis=1, errors='ignore')
    X_test = test_data_na_dropped.drop(all_learning_tasks_in_data, axis=1, errors='ignore')
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    # Fill with 0 the values that are missing in the input features so that the sample can still be used
    print('Filling NA in samples with ', fill_na_val)
    X_train = X_train.fillna(fill_na_val)
    X_test = X_test.fillna(fill_na_val)

    if (learning_task in ['delay_trace.txt_ul_delay', 'delay_trace.txt_dl_delay']) and clip_outliers:
        print('NOTE: clipping all rows with delay values > ', delay_clip_th)
        y_train.loc[y_train > delay_clip_th] = delay_clip_th
        y_test.loc[y_test > delay_clip_th] = delay_clip_th
        #print('NOTE: dropping all rows with delay values > ', delay_drop_th)
        #data = data[data['owd_ul'] <= delay_drop_th]
    
    # Save the columns to use for feature importance graphs
    X_feats = np.array(X_train.columns)
    np.savetxt('input_feature_list.csv', X_feats, delimiter=',', fmt="%s")

    # Convert everything to numpy 
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    # If you want to shift the output feature window 
    # I am taking away this option since we are combining multiple runs and the values at the edge between 2 runs will be wrong
    # I need to shift before combining runs and drop the last sample if I want to do this here 
    #if shift_samp_for_predict: 
    #    y = np.roll(y,1)
    #    X = X[1:]
    #    y = y[1:]   

    strat = None

    # Train test split
    # Random shift
    #sample_shift_count = np.random.randint(0, X.shape[0], size=1)
    #X = np.roll(X, sample_shift_count)
    #y = np.roll(y, sample_shift_count)
    
    # Randomly assign to train and test sets 
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, 
    #                                                    random_state=IN_PARAM['rand_seed'], stratify=strat) 
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, 
                                                  test_size=0.25, shuffle=True, 
                                                  random_state=IN_PARAM['rand_seed'],
                                                  stratify=strat)  
    
    print('train data shape ' + str(X_train.shape))
    print('test data shape ' + str(X_test.shape))
    print('val data shape ' + str(X_val.shape))
    
    # Plot hist of output
    # If classification it will just bin it
    #plt.figure(1)
    #plt.hist(y_train, bins=50, color='r', edgecolor='k', label='train samples')
    #plt.xlabel(learning_task)
    #plt.title('Histogram of train and test samples')
    #plt.legend()
    #plt.show()
    #plt.figure(2)
    #plt.hist(y_test, bins=50, color='b', edgecolor='k', label='test_samples')
    #plt.xlabel(learning_task)
    #plt.title('Histogram of train and test samples')
    #plt.legend()
    #plt.show()
    
    #=============================================== Train and test the model ==================================

    OUT_PARAM = dict.fromkeys(OUT_PARAM, 0)
    start_time = time.time()


    if use_pretrained_model:
        # get the latent representations from pretrained model
        # Load the pretrained model and scaler 
        # Load the saved MinMaxScaler object from the file
        with open(model_save_path + scaler_to_load_name + '.pkl', 'rb') as f:
            val_scaler = pickle.load(f)
            
        X_train = val_scaler.transform(X_train).copy()
        X_test = val_scaler.transform(X_test).copy()
        X_val = val_scaler.transform(X_val).copy()
        
        print('Before passing through pretrained model')
        print(X_train.shape)
        print(X_train)
        print('Number of rows with NAs in the train set: ', np.sum(np.any(np.isnan(X_train), axis=1)))
        print('Number of rows with NAs in the val set: ', np.sum(np.any(np.isnan(X_val), axis=1)))
        
        # Load the pretrained model
        if pretrain_model_to_load_type == 'dae':
            pretrain_model = dae_load(model_save_path+pretrain_model_to_load_name)
            X_train = pretrain_model.transform(pd.DataFrame(X_train))#[:, -hid_size:]
            X_test = pretrain_model.transform(pd.DataFrame(X_test))#[:, -hid_size:]
            X_val = pretrain_model.transform(pd.DataFrame(X_val))#[:, -hid_size:]
        elif pretrain_model_to_load_type == 'tabnet':
            pretrain_model = TabNetPretrainer()
            pretrain_model.load_model(model_save_path+pretrain_model_to_load_name)
            X_train = pretrain_model.predict(X_train)[0]
            X_test = pretrain_model.predict(X_test)[0]
            X_val = pretrain_model.predict(X_val)[0]
        else:
            print('Do not know model type')
        
        print('After pretrainer transform')
        print(X_train)
        print(X_train.shape)
        
        #print('Fraction of rows with NA values in the input features: ', len(X_pretrain[X_pretrain.isna().any(axis=1)])/X_pretrain.shape[0])

        print('Number of rows with NAs in the train set: ', np.sum(np.any(np.isnan(X_train), axis=1)))
        print('Number of rows with NAs in the val set: ', np.sum(np.any(np.isnan(X_val), axis=1)))
        print(X_train)
        print(X_train.shape)
        
    else: # only supervised training 
        val_scaler = create_scaler(X_train, EXP_PARAM['scaler'])
        # save the MinMaxscaler model to a file
        with open(model_save_path + scaler_to_save_name + '.pkl', 'wb') as f:
            pickle.dump(val_scaler, f)
        
        X_train = val_scaler.transform(X_train).copy()
        X_test = val_scaler.transform(X_test).copy()
        X_val = val_scaler.transform(X_val).copy()
    
    if load_sup_model:
        # Load the supervised trained model
        if sup_model_to_load_type == 'mlp':
            with open(model_save_path+sup_model_to_load_name+'.pkl', 'rb') as file:
                model = pickle.load(file)
        elif sup_model_to_load_type == 'tabnet':
            model = TabNetRegressor()
            model.load_model(model_save_path+sup_model_to_load_name+'.zip')
            # Feature importance 
            train_dataloader, valid_dataloaders = model._construct_loaders(X_train, y_train, list(zip(X_test, y_test)))
            plot_feature_importance(model._compute_feature_importances(train_dataloader), X_feats, feat_filter)
            #tabnet_explain(model, X_test, feat_filter, X_feats)
        elif sup_model_to_load_type == 'xgb':
            model = XGBRegressor()
            model.load_model(model_save_path+sup_model_to_load_name+'.json')
            # Feature importance 
            plot_feature_importance(model.feature_importances_, X_feats, feat_filter)
        else:
            print('Do not know model type')
    else: # Need to train the model
        # Train and save the model         
        model, history = train_model(X_train, X_val, 
                                        y_train, y_val, 
                                        IN_PARAM,
                                        model_save_path + suptrain_model_to_save_name,
                                        sup_hyper_params)
        end_time = time.time()
        OUT_PARAM['runtime'] = end_time - start_time
        print('Time to train model: ', end_time - start_time)   
        # Feature importance
        if (IN_PARAM['model_type'] == 'tabnet') or (IN_PARAM['model_type'] == 'xgb'):
            plot_feature_importance(model.feature_importances_, X_feats, feat_filter)
    
    #====================================
    # Predict using the supervised model 
    #====================================
    
    yhat_test = model.predict(X_test).flatten()
    yhat_train = model.predict(X_train).flatten()
    
    print(IN_PARAM['eval_metric'])
    OUT_PARAM['train_err'] = compute_error(y_train, yhat_train, IN_PARAM['eval_metric'])
    OUT_PARAM['test_err'] = compute_error(y_test, yhat_test, IN_PARAM['eval_metric'])
    print(OUT_PARAM)
    print('ML model: ', IN_PARAM['eval_metric'],' err for test set: ', OUT_PARAM['test_err'])
    print('Mean of the train set is: ', np.mean(y_train))
    print('Median of the train set is: ', np.median(y_train))

    print(y_test)
    print(yhat_test)
    bounds=[min( min(y_test),min(yhat_test) ), max( max(y_test),max(yhat_test) )]
    print(bounds)
    # Q-Q plot for the prediction and ground truth 
    plt.figure(figsize=(10, 10))
    plt.plot(y_test, yhat_test, 'b.')
    #plt.plot([0,0], [delay_drop_th, delay_drop_th], 'k-')
    plt.xlabel('Ground truth test samples')
    plt.ylabel('Prediction test samples')
    plt.xlim(bounds)
    plt.ylim(bounds)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.plot(bounds, bounds, 'k-')
    plt.grid()
    plt.show()
    
    if not os.path.isdir(model_save_path):
        os.makedirs(model_save_path)
    
            
    print('')
    print('')
    print('===============================  DONE  ===================================')

Learning task:  vrFragment_trace.txt_vr_frag_thput_mbps
Dropping rows for which the label is NA, since there is no ground truth
(39613, 215)
(9904, 215)
(39613, 207)
(39613,)
(9904, 207)
(9904,)
Filling NA in samples with  0
train data shape (39613, 207)
test data shape (7428, 207)
val data shape (2476, 207)


NameError: name 'StandardScaler' is not defined

# Some potentially useful code for later

In [ ]:
# plot the cdf of the train error 
print(yhat_train.shape) # ,1
print(y_train.shape) # ,
yhat_train_a = np.squeeze(yhat_train)
yhat_test_a = np.squeeze(yhat_test)
ecdf_train = ECDF(yhat_train_a - y_train)
plt.step(ecdf_train.x, ecdf_train.y)
plt.axvline(x=0, color='red', linestyle='--')
plt.axhline(y=ecdf_train(0), color='red', linestyle='--')
plt.xlabel('Pred err (truth - pred)')
plt.title('Train samples')
#plt.hist((yhat_train - y_train), bins=200, edgecolor='k')
#plt.xlim(-20, 50)
plt.show()
print('Train: Probability mass of pred err (truth-pred) below 0 is: ',  ecdf_train(0))
print('Train: Probability mass of pred err (truth-pred) above 0 is: ',  1-ecdf_train(0))

# plot the cdf of the test error 
ecdf_test = ECDF(yhat_test_a - y_test)
plt.step(ecdf_test.x, ecdf_test.y)
plt.axvline(x=0, color='red', linestyle='--')
plt.axhline(y=ecdf_test(0), color='red', linestyle='--')
plt.xlabel('Pred err (truth - pred)')
plt.title('Test samples')
#plt.hist((yhat_test - y_test), bins=200, edgecolor='k')
#plt.xlim(-20, 50)
plt.show()
print('Test: Probability mass of pred err (truth-pred) below 0 is: ',  ecdf_test(0))
print('Test: Probability mass of pred err (truth-pred) above 0 is: ',  1-ecdf_test(0))

#===================================== plot sorted samples of prediction overlayed with ground truth  ==========================
# 
#if sort_test_samples:   
#    #train_baseline_vals = np.repeat(baseline_pred, len(y_train))
#    #test_baseline_vals = np.repeat(baseline_pred, len(y_test))
#
#    tmp1 = np.append(np.expand_dims(y_train, axis=1), np.expand_dims(yhat_train, axis=1), axis=1)
#    tmp1 = tmp1[tmp1[:, 0].argsort()]
#    y_train = tmp1[:,0]
#    yhat_train = tmp1[:,1]
#
#    tmp2 = np.append(np.expand_dims(y_test, axis=1), np.expand_dims(yhat_test, axis=1), axis=1)
#    tmp2 = tmp2[tmp2[:, 0].argsort()]
#    y_test = tmp2[:,0]
#    yhat_test = tmp2[:,1]

##=============================================== bin the delay values to observe err per bin ==================================
##
## bin index for each delay value, so that we can put the values in the right bin 
#bin_indices = np.digitize(y_train, bin_edges)
#
## I want to take all the delay values for each bin
#for bin_ind in np.unique(bin_indices):
#    # these are the delay values in bin bin_edges[bin_ind]
#    train_bin_uldelay_mean[bin_ind-1] = train_bin_uldelay_mean[bin_ind-1] + np.sum(y_train[bin_indices == bin_ind])
#    train_bin_count[bin_ind-1] = train_bin_count[bin_ind-1] + len(y_train[bin_indices == bin_ind])
#    
#    # I want the corresponding err values for these delay values  
#    train_bin_err_mean[bin_ind-1] = (train_bin_err_mean[bin_ind-1] + 
#                                     np.sum(np.abs(y_train[bin_indices == bin_ind] - yhat_train[bin_indices == bin_ind]) ))
#    train_bin_baseline_err_mean[bin_ind-1] = (train_bin_baseline_err_mean[bin_ind-1] + 
#                                             np.sum(np.abs(y_train[bin_indices == bin_ind] - train_baseline_vals[bin_indices == bin_ind]) ))
#    train_bin_perc_err_mean[bin_ind-1] = (train_bin_perc_err_mean[bin_ind-1] + 
#                                          np.sum(np.abs((y_train[bin_indices == bin_ind] - yhat_train[bin_indices == bin_ind]))/(y_train[bin_indices == bin_ind]) ))
#    train_bin_baseline_perc_err_mean[bin_ind-1] = (train_bin_baseline_perc_err_mean[bin_ind-1] + 
#                                                  np.sum(np.abs((y_train[bin_indices == bin_ind] - train_baseline_vals[bin_indices == bin_ind]))/(y_train[bin_indices == bin_ind])) )
#
## bin index for each delay value
#bin_indices = np.digitize(y_test, bin_edges)
#
## I want to take all the delay values for each bin 
#for bin_ind in np.unique(bin_indices):
#    # these are the delay values in bin bin_edges[bin_ind]
#    test_bin_uldelay_mean[bin_ind-1] = test_bin_uldelay_mean[bin_ind-1] + np.sum(y_test[bin_indices == bin_ind])
#    test_bin_count[bin_ind-1] = test_bin_count[bin_ind-1] + len(y_test[bin_indices == bin_ind])
#    
#    # I want the corresponding err values for these delay values
#    test_bin_err_mean[bin_ind-1] = (test_bin_err_mean[bin_ind-1] + 
#                                    np.sum(np.abs(y_test[bin_indices == bin_ind] - yhat_test[bin_indices == bin_ind])) )
#    test_bin_baseline_err_mean[bin_ind-1] = (test_bin_baseline_err_mean[bin_ind-1] + 
#                                             np.sum(np.abs(y_test[bin_indices == bin_ind] - test_baseline_vals[bin_indices == bin_ind])))
#    test_bin_perc_err_mean[bin_ind-1] = ( test_bin_perc_err_mean[bin_ind-1] + 
#                                         np.sum(np.abs((y_test[bin_indices == bin_ind] - yhat_test[bin_indices == bin_ind]))/(y_test[bin_indices == bin_ind])) )
#    test_bin_baseline_perc_err_mean[bin_ind-1] = (test_bin_baseline_perc_err_mean[bin_ind-1] + 
#                                                  np.sum(np.abs((y_test[bin_indices == bin_ind] - test_baseline_vals[bin_indices == bin_ind]))/(y_test[bin_indices == bin_ind])) ) 
#
## Plot
#plot_y_yhat(y_train, yhat_train, y_test, yhat_test, model_save_path, IN_PARAM)
#
## Convert the regression output to class labels and do confusion matrix
#cf_matrix = confusion_matrix(value_to_class_label(y_test, delay_class_edges), 
#                             value_to_class_label(yhat_test, delay_class_edges), normalize='true')
#sns.set(rc={'figure.figsize':(8,7)}, font_scale = 1.5)
#sns.heatmap(cf_matrix, annot=True, 
#    fmt='.1%', cmap='Blues')
#
#
#=============================================== plot q-q prediction versus ground truth  ==================================


# Binning by Y vale to see if the prediction error in each bin is consistent or not 

print('-----------------------------------------------------------')
print('Top n feature list size: ', len(top_n_features))
print(top_n_features)
print('-----------------------------------------------------------')

print('Loss fun: ', IN_PARAM['loss'])
print('Eval err fun: ', IN_PARAM['eval_metric'])
# After going over all runs     
fig, ax1 = plt.subplots(figsize=(16, 4))
ax2 = ax1.twinx()
ax1.plot(train_bin_uldelay_mean/train_bin_count, train_bin_err_mean/train_bin_count, 'r.-', label='XGB pred err (truth-pred)')
#ax1.plot(train_bin_uldelay_mean/train_bin_count, train_bin_baseline_err_mean/train_bin_count, 'm.-', label='baseline pred err (truth-pred)')
ax1.axhline(y=0, color='r', linestyle='--')
ax1.set_ylabel('error')
#ax1.set_ylim(-5,25)
ax1.legend()
plt.xlabel('uplink delay (ms)')
plt.title('Train samples')
plt.grid()
ax2.set_ylabel('relative err')
ax2.plot(train_bin_uldelay_mean/train_bin_count, train_bin_perc_err_mean/train_bin_count, 'g*-', label='XGB relative err')
#ax2.plot(train_bin_uldelay_mean/train_bin_count, train_bin_baseline_perc_err_mean/train_bin_count, 'c*-', label='baseline relative err')
ax2.axhline(y=0, color='g', linestyle='--')
plt.xscale('log')
ax1.legend(loc=6)
ax2.legend(loc=1)
plt.show() 

plt.figure(figsize=(16, 2))
plt.plot(train_bin_uldelay_mean/train_bin_count, train_bin_count, 'b*-')
plt.xlabel('Train samples uplink delay bin (ms)')
plt.xscale('log')
plt.ylabel('bin count')
plt.grid()
plt.show()

print('Loss fun: ', IN_PARAM['loss'])
print('Eval err fun: ', IN_PARAM['eval_metric'])
fig, ax1 = plt.subplots(figsize=(16, 4))
ax2 = ax1.twinx()
ax1.plot(test_bin_uldelay_mean/test_bin_count, test_bin_err_mean/test_bin_count, 'r.-', label='XGB pred err (truth-pred)')
#ax1.plot(test_bin_uldelay_mean/test_bin_count, test_bin_baseline_err_mean/test_bin_count, 'm.-', label='baseline pred err (truth-pred)')
ax1.axhline(y=0, color='r', linestyle='--')
plt.xlabel('uplink delay (ms)')
ax1.set_ylabel('error')
#ax1.set_ylim(-500,250)
ax1.legend()
plt.title('Test samples')
plt.grid()
ax2.set_ylabel('relative err')
ax2.plot(test_bin_uldelay_mean/test_bin_count, test_bin_perc_err_mean/test_bin_count, 'g*-', label='relative err')
#ax2.plot(test_bin_uldelay_mean/test_bin_count, test_bin_baseline_perc_err_mean/test_bin_count, 'c*-', label='baseline relative err')
ax2.axhline(y=0, color='g', linestyle='--')
plt.xscale('log')
ax1.legend(loc=6)
ax2.legend(loc=1)
plt.show()

plt.figure(figsize=(16, 2))
plt.plot(test_bin_uldelay_mean/test_bin_count, test_bin_count, 'b*-')
plt.xlabel('Test samples uplink delay bins (ms)')
plt.ylabel('bin count')
plt.xscale('log')
plt.grid()
plt.show()


In [17]:
notebook_save_str = 'no_pretrain_tabnet_notebook'
os.system('cp train_and_eval_ML_models.ipynb '+'../saved_notebooks/'+notebook_save_str+'.ipynb')

0

In [ ]:
feat_union = ['ul_loadUe_weightBand', 'ul_loadUe_BSRestimate', 'ul_BSRestimateLcg6', 'ul_loadUe_deltaIcc', 'ul_nrOfCsiPart1Bits', 'ul_loadUe_RI', 'dl_pucchFormatType', 'ul_loadUe_BSValueLcg6', 'ul_rv', 'ul_tbSizeInBits', 'dl_loadUe_dciFormat', 'dl_loadUe_pucchResourceIndicator', 'nbm_loadUe_WBeamIndex', 'ul_loadUe_fda', 'dl_NBeamRsrpCurrent', 'dl_loadUe_transmissionAttempt', 'dl_loadUe_bbBearerRef7', 'ul_BSRestimateLcg4', 'dl_numOfPrbs', 'ul_loadUe_isTransformPrecoding', 'ul_loadUe_timingOffset', 'dl_numberOfSymbols', 'ul_BSValueLcg0', 'dl_tbSizeInBits', 'ul_loadUe_VBit', 'ul_loadUe_BSRestimateLcg2', 'ul_csiRequest', 'ul_loadUe_BSValueLcg4', 'dl_loadUe_weightBand', 'wbm_rsrp1', 'dl_loadUe_NBeamRsrpCurrent', 'ul_tpcCommand', 'ul_iccAchievable', 'ul_loadUe_preamblePwr', 'nbm_beam03', 'dl_drb5data', 'dl_pucchSfn', 'ul_loadUe_numOfLayers', 'nbm_rsrp01', 'dl_icc', 'ul_loadUe_iccAchievable', 'ul_fda', 'ul_loadUe_uciDecodedResult', 'ul_loadUe_BSValueLcg0', 'dl_dlCcIndex', 'nbm_loadUe_rsrp11', 'wbm_beam4', 'dl_loadUe_incrementalWeight', 'dl_loadUe_antennaPorts', 'ul_BSValueLcg5', 'ul_RI', 'dl_loadUe_fda', 'dl_loadUe_numBearers', 'ul_loadUe_csiRequest', 'dl_linkAdaptationUeMode', 'wbm_loadUe_cellId', 'ul_loadUe_ACK', 'dl_cellId', 'dl_bbBearerRef2', 'ul_loadUe_BSValueLcg7', 'dl_loadUe_cqi', 'dl_pdschHarqFeedbackTiming', 'dl_loadUe_pucchSfn', 'dl_loadUe_pdschHarqFeedbackTiming', 'ul_loadUe_isClpcSaturated', 'ul_loadUe_startSymbolPusch', 'dl_loadUe_dlCcIndex', 'dl_loadUe_drb7input', 'dl_bbBearerRef7', 'dl_loadUe_bbBearerRef2', 'ul_loadUe_beamIndex', 'dl_rank', 'dl_loadUe_drb4input', 'dl_numBearers', 'wbm_loadUe_beam2', 'nbm_loadUe_beam01', 'ul_loadUe_WBeamRsrpCurrent', 'ul_carrierAggregationUsed', 'nbm_loadUe_rsrp12', 'dl_drb4input', 'dl_mcsIndex', 'dl_sinr', 'dl_drb4data', 'ul_loadUe_harqProcessId', 'dl_transmissionAttempt', 'ul_loadUe_BSRestimateLcg7', 'dl_loadUe_isDrxEnabled', 'dl_drb0data', 'dl_loadUe_numOfPrbs', 'dl_loadUe_drb4data', 'ul_loadUe_numberOfSymbolsPusch', 'nbm_loadUe_ueTraceIdMsw', 'wbm_loadUe_beam3', 'dl_sectorIndex', 'dl_drb1input', 'dl_numberOfSrcBits', 'nbm_beam13', 'ul_clpcCarrierDemand', 'ul_wbUsedNbOverridden', 'ul_loadUe_puschTotalRxPsdAvg', 'dl_srOnPucch', 'dl_loadUe_sectorIndex', 'wbm_beam3', 'dl_drb0input', 'ul_BSRestimate', 'ul_BSRestimateLcg1', 'dl_loadUe_taValue', 'ul_maxRank', 'wbm_beam1', 'dl_loadUe_bbBearerRef3', 'dl_physicalCellId', 'ul_totalCellsReqScheduling', 'wbm_loadUe_beam4', 'nbm_loadUe_rsrp02', 'wbm_loadUe_WBeamIndexNewBest', 'dl_loadUe_WBeamRsrpCurrent', 'nbm_WBeamIndex', 'dl_loadUe_slot', 'dl_bbBearerRef1', 'dl_drb3data', 'nbm_loadUe_rsrp13', 'ul_macSduInBytes', 'dl_loadUe_srOnPucch', 'ul_measNumOfPrb', 'ul_transmissionAttempt', 'ul_loadUe_BSRestimateLcg3', 'dl_loadUe_deltaIcc', 'dl_dciFormat', 'ul_bbCellIndex', 'dl_loadUe_macCtrlElement', 'dl_weightBand', 'ul_loadUe_mcsIndex', 'ul_loadUe_antennaPorts', 'ul_timingOffset', 'dl_redundancyVersion', 'ul_loadUe_BSRestimateLcg1', 'ul_loadUe_wbUsedNbOverridden', 'dl_numberOfActivatedDlCells', 'dl_antennaPorts', 'dl_drb7input', 'dl_harqProcessId', 'nbm_loadUe_beam13', 'ul_loadUe_nrOfCsiPart1Bits', 'ul_dciFormat', 'dl_pucchSlotNo', 'nbm_NBeamIndexChosen', 'ul_precodingInfo', 'ul_loadUe_NBeamRsrpCurrent', 'ul_loadUe_harqFailure', 'dl_bbBearerRef4', 'dl_ndi', 'nbm_beam12', 'dl_loadUe_tda', 'nbm_loadUe_cellId', 'wbm_loadUe_WBeamIndexCurrent', 'ul_ndi', 'dl_loadUe_tbSizeInBits', 'wbm_cellId', 'dl_loadUe_icc', 'nbm_loadUe_beam02', 'dl_pucchResourceIndicator', 'ul_BSValueLcg2', 'ul_loadUe_macSduInBytes', 'ul_measNumOfLayers', 'dl_loadUe_drb1input', 'dl_WBeamRsrpCurrent', 'nbm_loadUe_noOfCriPerCsiReport', 'ul_antennaPorts', 'ul_loadUe_slot', 'wbm_beam2', 'ul_loadUe_startPrb', 'ul_slot', 'nbm_loadUe_rsrp01', 'ul_BSRestimateLcg0', 'ul_loadUe_numOfPrbs', 'ul_isDrxEnabled', 'ul_loadUe_BSRestimateLcg0', 'wbm_rsrp4', 'ul_BSValueLcg4', 'dl_loadUe_pucchSlotNo', 'ul_preamblePwr', 'wbm_loadUe_beam1', 'dl_ACK', 'ul_isPrimaryCell', 'dl_tda', 'ul_loadUe_postEqSinr0', 'dl_loadUe_sinr', 'ul_loadUe_pCmaxCIndex', 'ul_loadUe_cellId', 'dl_loadUe_physicalCellId', 'dl_loadUe_beamIndex', 'dl_loadUe_numberOfSymbols', 'dl_loadUe_drb2data', 'dl_loadUe_drb1data', 'ul_BSRestimateLcg7', 'ul_numOfLayers', 'ul_loadUe_numberOfActivatedUlCells', 'ul_loadUe_BSValueLcg1', 'nbm_beam11', 'ul_loadUe_BSValueLcg5', 'dl_loadUe_newData/reTx(1/0)', 'dl_loadUe_bbBearerRef6', 'ul_loadUe_bbCellIndex', 'nbm_loadUe_NBeamIndexChosen', 'dl_loadUe_numberOfActivatedDlCells', 'ul_isClpcSaturated', 'ul_loadUe_BSRestimateLcg5', 'ul_loadUe_tbSizeInBits', 'ul_BSValueLcg1', 'ul_loadUe_maxRank', 'dl_fda', 'ul_DTX', 'ul_loadUe_clpcCarrierDemand', 'dl_cqi', 'dl_dai', 'wbm_rsrp3', 'ul_numOfPrbs', 'dl_drb3input', 'ul_puschTotalRxPsdAvg', 'nbm_loadUe_NBeamIndexCurrent', 'nbm_cellId', 'ul_loadUe_postEqSinr1', 'dl_loadUe_drb3input', 'ul_pCmaxCIndex', 'ul_numberOfSymbolsPusch', 'dl_loadUe_mcsIndex', 'dl_newData/reTx(1/0)', 'ul_weightBand', 'nbm_rsrp02', 'ul_uciDecodedResult', 'ul_loadUe_BSValueLcg2', 'ul_powerHeadRoomIndex', 'ul_loadUe_rv', 'ul_incrementalWeight', 'dl_loadUe_drb2input', 'dl_loadUe_drb7data', 'dl_drb6input', 'wbm_rsrp2', 'dl_loadUe_bbCellIndex', 'dl_bbBearerRef3', 'dl_loadUe_feedbackIndex', 'ul_startPrb', 'ul_cellId', 'wbm_loadUe_rsrp1', 'dl_loadUe_rank', 'ul_ACK', 'ul_beamIndex', 'dl_isSrBitIncluded', 'ul_numberOfActivatedUlCells', 'dl_bbBearerRef6', 'ul_loadUe_transmissionAttempt', 'dl_loadUe_isPrimaryCell', 'wbm_loadUe_rsrp4', 'ul_BSValueLcg3', 'dl_macCtrlElement', 'dl_drb6data', 'dl_isPrimaryCell', 'ul_harqFailure', 'dl_bbCellIndex', 'ul_loadUe_totalCellsReqScheduling', 'ul_postEqSinr0', 'ul_loadUe_carrierAggregationUsed', 'dl_loadUe_bbBearerRef4', 'ul_loadUe_powerHeadRoomIndex', 'ul_BSValueLcg6', 'nbm_loadUe_beam03', 'dl_drb2data', 'nbm_rsrp13', 'dl_isDrxEnabled', 'ul_loadUe_measNumOfPrb', 'dl_feedbackIndex', 'ul_loadUe_isDrxEnabled', 'dl_loadUe_bbBearerRef1', 'ul_loadUe_ulRequestTypeBitmap', 'nbm_rsrp03', 'dl_slot', 'dl_loadUe_drb5input', 'ul_loadUe_BSRestimateLcg4', 'dl_beamIndex', 'dl_loadUe_isSrBitIncluded', 'ul_BSRestimateLcg3', 'ul_WBeamRsrpCurrent', 'dl_loadUe_DTX', 'nbm_rsrp11', 'ul_BSRestimateLcg5', 'dl_loadUe_redundancyVersion', 'ul_ulRequestTypeBitmap', 'wbm_WBeamIndexNewBest', 'ul_ulschIndicator', 'dl_drb1data', 'dl_drb2input', 'ul_loadUe_BSRestimateLcg6', 'nbm_loadUe_rsrp03', 'dl_loadUe_wbUsedNbOverridden', 'dl_loadUe_drb6input', 'nbm_loadUe_beam11', 'nbm_beam01', 'dl_loadUe_linkAdaptationUeMode', 'ul_loadUe_linkAdaptationUeMode', 'dl_loadUe_bbBearerRef0', 'dl_drb5input', 'dl_loadUe_drb6data', 'ul_loadUe_BSValueLcg3', 'dl_loadUe_drb0data', 'ul_loadUe_sinrAchievable', 'nbm_loadUe_beam12', 'ul_loadUe_incrementalWeight', 'dl_loadUe_dai', 'dl_loadUe_ueTraceIdMsw', 'ul_loadUe_ndi', 'ul_startSymbolPusch', 'dl_loadUe_cellId', 'ul_isTransformPrecoding', 'dl_loadUe_drb5data', 'dl_loadUe_drb3data', 'ul_deltaIcc', 'nbm_noOfCriPerCsiReport', 'dl_loadUe_drb0input', 'ul_mcsIndex', 'ul_chipsetType', 'dl_loadUe_numberOfSrcBits', 'nbm_beam02', 'ul_postEqSinr1', 'ul_loadUe_isPrimaryCell', 'dl_incrementalWeight', 'dl_loadUe_ACK', 'ul_BSRestimateLcg2', 'ul_NBeamRsrpCurrent', 'dl_DTX', 'dl_bbBearerRef5', 'dl_loadUe_bbBearerRef5', 'ul_harqProcessId', 'dl_loadUe_pucchFormatType', 'wbm_loadUe_rsrp2', 'dl_taValue', 'ul_loadUe_precodingInfo', 'ul_BSValueLcg7', 'wbm_loadUe_ueTraceIdMsw', 'dl_loadUe_ndi', 'dl_deltaIcc', 'dl_drb7data', 'ul_loadUe_DTX', 'ul_linkAdaptationUeMode', 'wbm_loadUe_rsrp3', 'nbm_NBeamIndexCurrent', 'dl_loadUe_harqProcessId', 'ul_loadUe_ulschIndicator', 'dl_wbUsedNbOverridden', 'ul_loadUe_measNumOfLayers', 'nbm_rsrp12', 'dl_bbBearerRef0', 'wbm_WBeamIndexCurrent', 'ul_sinrAchievable']
print(feat_union)
print(len(feat_union))

In [ ]:
# List of featurtes I think will not generalize 
filtered_cols = [i for i in feat_union if 'eamIndex' in i]
print(filtered_cols)